In [1]:
import os
import os.path
import selenium
from selenium import webdriver
import time
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import numpy as np
import csv
from selenium.webdriver.support.ui import WebDriverWait
import datetime
from datetime import date, timedelta
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import pickle

In [33]:
# Variables PL : A faire tourner pour scrapper la PL

# Ligue
league_str = "a[href='/tournament/football/england/premier-league/17']"
# Les matchs (format : a>div>div.Cell-sc ...)
match_str = 'a>div>div.Cell-sc-t6h3ns-0'
# La date (format : div>div.classe)
date_str = 'div>div.ixemiF'
# Le nom des équipes (format : div.classe > div.Content-...)
team_names_str = 'div.bGpZoa>div.Content-sc-1morvta-0'
# Le score
score_str = 'div.jUEsho>div.Content-sc-1morvta-0'
# Le button stat(format : .classe)
buttons_str = ".iCdnqS"
# Les stats (format : .styles__StatisticsItemContent...)
stats_str = '.styles__StatisticsItemContent-sc-1imujgi-0'
# Croix (format : path[d:'XX X XXX ...'])
cross_str = "path[d='M4 4 L20 20 M4 20 L20 4']"

In [2]:
# Variables Ligue1 : A faire tourner pour scrapper la Ligue 1

# Ligue
league_str = "a[href='/tournament/football/france/ligue-1/34']"
# Les matchs (format : a>div>div.Cell-sc ...)
match_str = 'a>div>div.Cell-sc-t6h3ns-0'
# La date (format : div>div.classe)
date_str = 'div>div.ixemiF'
# Le nom des équipes (format : div.classe > div.Content-...)
team_names_str = 'div.bGpZoa>div.Content-sc-1morvta-0'
# Le score
score_str = 'div.jUEsho>div.Content-sc-1morvta-0'
# Le button stat(format : .classe)
buttons_str = ".iCdnqS"
# Les stats (format : .styles__StatisticsItemContent...)
stats_str = '.styles__StatisticsItemContent-sc-1imujgi-0'
# Croix (format : path[d:'XX X XXX ...'])
cross_str = "path[d='M4 4 L20 20 M4 20 L20 4']"

In [35]:
# Scrapper

#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

#Specify Search URL
search_url='https://www.sofascore.com/'


filehandler = open ('filename.pickle', 'w') 


def daterange(start_date, end_date,filehandler):
    with open('teams_stats_sofascore_L1.csv','wb') as f:
        dic_matchs = {}
        i = 0
        ligue_7079 = []
        for n in range(int((end_date - start_date).days)):
            date = start_date + timedelta(n)
            search_url = f"https://www.sofascore.com/{date}"
            driver.get(search_url)
            ligue_tags = driver.find_elements_by_css_selector(league_str)
            if len(ligue_tags) < 2:
                driver.execute_script("window.scrollBy(0, arguments[0]);", 1000)
                ligue_tags = driver.find_elements_by_css_selector(league_str)
            if len(ligue_tags) >= 2:
                ligue_tag = ligue_tags[1].find_element_by_xpath('..').find_element_by_xpath('..').find_element_by_xpath('..').find_element_by_xpath('..')
                test_tag = ligue_tags[1].find_element_by_xpath('..').find_element_by_xpath('..').find_element_by_xpath('..').find_element_by_xpath('..')
                ligue_tag_style = ligue_tag.get_attribute("style")
                test_tag_style = ligue_tag.get_attribute("style")
                matchs=[]
                matchs_childs=[]
                while test_tag_style[:12] == ligue_tag_style[:12]:
                    matchs.append(test_tag)
                    #Si jamais le tag de Ligue 1 est trop bas
                    if test_tag.location['y'] > 1000 :
                        desired_y1 = (test_tag.size['height'] / 2) + test_tag.location['y']
                        current_y1 = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
                        scroll_y1_by = desired_y1 - current_y1
                        driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y1_by)
                    #time.sleep(2)
                    test_tag_style = test_tag.find_element_by_xpath("following-sibling::div").get_attribute('style')
                    test_tag = test_tag.find_element_by_xpath("following-sibling::div")
                del matchs[0]
                for match in matchs:
                    if match:
                        matchs_childs.append(match.find_element_by_css_selector(f"{match_str}"))
                for game in matchs_childs:
                    dic_matchs[i] = {}
                    dic_matchs[i+1] = {}
                    date = game.find_element_by_css_selector(f"{date_str}")
                    team_names = game.find_elements_by_css_selector(f"{team_names_str}")
                    score = game.find_elements_by_css_selector(f"{score_str}")
                    dic_matchs[i]['Date'] = date.get_attribute('innerHTML')
                    dic_matchs[i+1]['Date'] = date.get_attribute('innerHTML')
                    dic_matchs[i]['Home'] = 1
                    dic_matchs[i+1]['Home'] = 0
                    dic_matchs[i]['Team'] = team_names[0].text
                    dic_matchs[i]['Opponent'] = team_names[1].text
                    dic_matchs[i+1]['Team'] = team_names[1].text
                    dic_matchs[i+1]['Opponent'] = team_names[0].text
                    dic_matchs[i]['Score team'] = score[0].text
                    dic_matchs[i]['Score opponent'] = score[1].text
                    dic_matchs[i+1]['Score team'] = score[1].text
                    dic_matchs[i+1]['Score opponent'] = score[0].text
                    desired_y = (game.size['height'] / 2) + game.location['y']
                    current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
                    scroll_y_by = desired_y - current_y
                    driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
                    game.click()
                    time.sleep(1)
                    buttons = driver.find_elements_by_css_selector(f"{buttons_str}")
                    buttons[2].click()
                    time.sleep(1)
                    stats = driver.find_elements_by_css_selector(f"{stats_str}")
                    ind = 1
                    ind_2 = 2
                    if len(stats) > 0 :
                        for stat in stats:
                            ind_stat = stats.index(stat)
                            data = stat.get_attribute('innerHTML')
                            if ind_stat not in [ind,ind_2]:
                                dic_matchs[i][f"{stats[ind_stat+1].get_attribute('innerHTML')}"] = data
                            elif ind_stat == ind_2:
                                dic_matchs[i+1][f"{stats[ind_stat-1].get_attribute('innerHTML')}"] = data
                                ind_2 = ind_2 + 3
                            else:
                                ind = ind + 3
                    i = i + 2             
                    time.sleep(1)
                    driver.find_element_by_css_selector(f"{cross_str}").click()
                    with open('filename.pickle', 'wb') as handle:
                        pickle.dump(dic_matchs, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return dic_matchs


start_date = datetime.date(2021,8,6)
end_date = datetime.date(2021,12,13)

global_dic = {}
# Last match of the league played date
ligue_date = datetime.date(2021, 12,12)

# Gestion des erreurs 

while start_date != ligue_date:
    try:
        daterange(start_date, end_date,filehandler)
        with open('filename.pickle', 'rb') as handle:
            dic_match = pickle.load(handle)
            i = 0
            for key, value in dic_match.items():
                global_dic[len(global_dic) + i] = value
                i = i + 1
            last_date_raw = dic_match[len(dic_match)-1]['Date']
            last_date = last_date_raw[:2] + '-' + last_date_raw[3:5]+ '-' + last_date_raw[6:]
            last_date_2 = datetime.datetime.strptime(last_date, '%d-%m-%y').date()
            start_date = last_date_2
    except:
        with open('filename.pickle', 'rb') as handle:
            dic_match = pickle.load(handle)
            i = 0
            for key, value in dic_match.items():
                global_dic[len(global_dic) + i] = value
                i = i + 1
            last_date_raw = dic_match[len(dic_match)-1]['Date']
            last_date = last_date_raw[:2] + '-' + last_date_raw[3:5]+ '-' + last_date_raw[6:]
            last_date_2 = datetime.datetime.strptime(last_date, '%d-%m-%y').date()
            start_date = last_date_2
            
            



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/charlesproye/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/7l/vp4hkxqx6m9901fw5fydkvww0000gn/T/ipykernel_52094/3076515292.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/7l/vp4hkxqx6m9901fw5fydkvww0000gn/T/ipykernel_52094/3076515292.py:23: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  ligue_tags = driver.find_elements_by_css_selector(league_str)
/Users/charlesproye/opt/miniconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element()

In [9]:
# L1 columns : A faire tourner pour scrapper la Ligue 1

# Have the columns in the right order for CSV
column_names_L1 = [
    'Date',
    'Home',
    'Team',
    'Opponent',
    'Score team',
    'Score opponent',
    'Ball possession',
    'Total shots',
    'Shots on target',
    'Shots off target',
    'Blocked shots',
    'Corner kicks',
    'Offsides',
    'Fouls',
    'Yellow cards',
    'Big chances',
    'Big chances missed',
    'Shots inside box',
    'Shots outside box',
    'Goalkeeper saves',
    'Passes',
    'Acc. passes',
    'Long balls',
    'Crosses',
    'Dribbles',
    'Possession lost',
    'Duels won',
    'Aerials won',
    'Tackles',
    'Interceptions',
    'Clearances',
    'Counter attacks',
    'Red cards',
    'Hit woodwork'
]

In [ ]:
# PL columns : : A faire tourner pour scrapper la PL

# Have the columns in the right order for CSV
column_names_PL = [
    'Date',
    'Home',
    'Team',
    'Opponent',
    'Score team',
    'Score opponent',
    'Ball possession',
    'Total shots',
    'Shots on target',
    'Shots off target',
    'Blocked shots',
    'Corner kicks',
    'Offsides',
    'Fouls',
    'Yellow cards',
    'Big chances',
    'Big chances missed',
    'Counter attacks',
    'Counter attack shots',
    'Counter attack goals'
    'Shots inside box',
    'Shots outside box',
    'Goalkeeper saves',
    'Passes',
    'Acc. passes',
    'Long balls',
    'Crosses',
    'Dribbles',
    'Possession lost',
    'Duels won',
    'Aerials won',
    'Tackles',
    'Interceptions',
    'Clearances',
    'Hit woodwork',
    'Red cards'
]


In [36]:
# To check 
dfL1 = pd.DataFrame.from_dict(global_dic, orient='index')
dfL1

,Date,Home,Team,Opponent,Score team,Score opponent,Ball possession,Total shots,Shots on target,Shots off target,...,Aerials won,Tackles,Interceptions,Clearances,Yellow cards,Counter attacks,Counter attack shots,Counter attack goals,Hit woodwork,Red cards
0,06/08/21,1,AS Monaco,FC Nantes,1,1,75%,15,3,9,...,15,20,11,5,NaN,NaN,NaN,NaN,NaN,NaN
2,06/08/21,0,FC Nantes,AS Monaco,1,1,25%,5,1,3,...,14,19,14,26,NaN,NaN,NaN,NaN,NaN,NaN
4,07/08/21,1,Olympique Lyonnais,Stade Brestois 29,1,1,69%,17,4,9,...,16,16,7,11,1,0,0,0,NaN,NaN
6,07/08/21,0,Stade Brestois 29,Olympique Lyonnais,1,1,31%,13,8,4,...,18,16,15,24,1,2,2,1,NaN,NaN
8,07/08/21,1,Troyes,Paris Saint-Germain,1,2,43%,15,7,4,...,2,17,8,14,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,12/12/21,0,Bordeaux,Troyes,2,1,35%,7,4,1,...,18,17,13,36,3,NaN,NaN,NaN,0,NaN
573,12/12/21,1,Strasbourg,Olympique de Marseille,0,2,49%,11,3,6,...,13,12,7,9,2,NaN,NaN,NaN,NaN,NaN
575,12/12/21,0,Olympique de Marseille,Strasbourg,2,0,51%,8,3,4,...,9,14,9,25,1,NaN,NaN,NaN,NaN,NaN
577,12/12/21,1,Paris Saint-Germain,AS Monaco,2,0,53%,9,2,4,...,8,13,16,12,3,2,1,1,1,NaN


In [37]:
# Sortie CSV L1 : A faire tourner pour scrapper la Ligue 1

# Transform into DF
#dfL1 = pd.DataFrame.from_dict(global_dic, orient='index')
# Index the columns to match the CSV template
dfL1 = dfL1.reindex(columns=column_names_L1)
# Create new CSC if first time with header
#dfL1.to_csv('sofascore_L1_20182021.csv',mode='w',index=False,header=True)
# Add to the CSV without the header and indexes
dfL1.to_csv('sofascore_L1_20182021.csv',mode='a',index=False,header=False)

In [ ]:
# Sortie CSV PL : A faire tourner pour scrapper la PL

# Transform into DF
dfPL = pd.DataFrame.from_dict(global_dic, orient='index')
# Index the columns to match the CSV template
#dfPL = dfPL.reindex(columns=column_names_PL)
# Add to the CSV without the header and indexes
dfPL.to_csv('sofascore_PL_20182021.csv',mode='a',index=False,header=True)
# Add to the CSV without the header and indexes
#dfPL.to_csv('sofascore_PL_20182021.csv',mode='a',index=False,header=False)